<font size=8>name</font>

# 项目说明

，。

# 数据说明

，。

# 建模目的

。、

# 依赖库及公共函数

## 依赖库

In [ ]:
"""
1/依赖库
"""
import math
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
from time import time
from random import seed, shuffle, sample, choice

# from sklearnex import patch_sklearn
# patch_sklearn()

from sklearn.metrics import make_scorer, accuracy_score, r2_score, log_loss, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, StratifiedKFold, StratifiedGroupKFold, RepeatedStratifiedKFold, TimeSeriesSplit
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

filterwarnings("ignore")
sns.set_style("ticks")
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 20)
pd.set_option('mode.chained_assignment', None)
plt.rcParams["figure.max_open_warning"] = 100
plt.rcParams['font.sans-serif']=['Simhei']
plt.rcParams['axes.unicode_minus'] = False

"""
2/全局变量设置
"""
random_state = 42
seed(random_state)
np.random.seed(random_state)

# 交叉验证
cv_skf = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
#cv_sgkf = StratifiedGroupKFold(n_splits=3, shuffle=True, random_state=random_state)
cv_sgkf = StratifiedGroupKFold(n_splits=3, shuffle=False)

# 模型评价
scoring_acc = make_scorer(accuracy_score, greater_is_better=True, needs_proba=False)
scoring_log = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

## 公共函数定义

### 建模相关

In [ ]:
"""
1/数据探索相关函数
"""
# 相关系数热力图矩阵
def plot_corr(corr, thresh=.0):
    kot = corr[abs(corr)>=thresh]
    plt.figure(figsize=(17, 17))
    #c_map = sns.diverging_palette(230, 20, as_cmap=True)
    c_map = 'coolwarm_r'
    sns.heatmap(kot, annot=True, fmt=".2f", cmap=c_map, square=True, center=0.0)
    plt.show()


# 特征数据分布
def plot_distribution(data,feature_names,figsize=(4.5, 4)):
    for name in feature_names:
        plt.figure(figsize=figsize)
        sns.histplot(x=name, data=data)
        plt.show()

def plot_distribution_multi(data,feature_names,figsize=(12,10)):
    # Plots the histogram for each numerical feature in a separate subplot
    df_X = data[feature_names]
    df_X.hist(bins=25, figsize=figsize, layout=(-1, 4), edgecolor="black")
    plt.tight_layout()
    plt.show()

def sns_distribution(data, feature_names, hue=None, col=None, figsize=(12,8)):
    #plt.rcParams['figure.figsize'] = figsize
    fig = plt.figure(figsize = figsize)
    #plt.style.use("ggplot")
    for name in feature_names:
        if hue:
            if col:
                sns.displot(data, x=name, hue=hue, col=col, kde=True)
            else:
                sns.displot(data, x=name, hue=hue, kde=True)
        else:
            sns.displot(data, x=name, kde=True)

def cmshow(mdl, X, y, label_names):
    y_hat= mdl.predict(X)
    test_score = evaluate_prediction(y, y_hat, accuracy_score, "test")
    print(f"准确率：{test_score}")

    conf_mat = confusion_matrix(y_true=y, y_pred=y_hat)

    cm_obj = ConfusionMatrixDisplay(conf_mat, display_labels=label_names)
    cm_obj.plot()
    plt.xlabel("预测结果")
    plt.ylabel("实际结果")
    plt.title("混淆矩阵：实际值 vs 预测值")
    #plt.savefig("Confusion_Matrix.png")
    plt.xticks(rotation=90)
    plt.show()

def checkmissing(df):
    missing = (df.isna().sum()/df.shape[0]*100).to_frame().reset_index().rename(columns = {'index':'column', 0:'pct_missing'})
    fig, ax = plt.subplots()
    ax = sns.barplot(data = missing, x = 'column', y = 'pct_missing')
    ax.set_ylabel('% Missing Values')
    ax.set_xlabel('Column Names')
    plt.xticks(rotation = 90)
    plt.show()

"""
2/建模相关函数
"""
# 分类模型预测准确率
def evaluate_prediction(y, y_hat, score_func, set_name="test"):
    score = score_func(y, y_hat)
    print(f"\n{set_name} score: {score}\n")
    print("\nconfusion_matrix:\n")
    print(confusion_matrix(y, y_hat))
    print("\nclassification_report:\n")
    print(classification_report(y, y_hat, digits=6))

    return score


# %% 学习曲线
def plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=None,
    ylim=None,
    cv=None,
    n_jobs=None,
    train_sizes=np.linspace(0.1, 1.0, 10),
    groups=None,
):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        X,
        y,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
        groups=groups,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    fit_time_argsort = fit_times_mean.argsort()
    fit_time_sorted = fit_times_mean[fit_time_argsort]
    test_scores_mean_sorted = test_scores_mean[fit_time_argsort]
    test_scores_std_sorted = test_scores_std[fit_time_argsort]
    axes[2].grid()
    axes[2].plot(fit_time_sorted, test_scores_mean_sorted, "o-")
    axes[2].fill_between(
        fit_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

# 交叉验证，超参优化
class SuperEstimator:
    def __init__(self, searchcv, multiple=0):
        self.estimator = searchcv.estimator
        self.searchcv = searchcv
        self.multiple = multiple

    def _best_mean_std(self, cv_results_):
        """
        基于验证集成绩的均值与标准差，算出一个综合的成绩。
        """
        scores = np.array(cv_results_["mean_test_score"]) - self.multiple * np.array(
            cv_results_["std_test_score"]
        )
        best_idx = np.argmax(scores)
        return best_idx

    def search(self, X, y=None, groups=None, verbose=True, n_jobs=-1):
        """
        超参优化。
        """
        t0 = time()
        self.searchcv.fit(X, y=y, groups=groups)
        t1 = time()
        best_idx = self._best_mean_std(self.searchcv.cv_results_)
        best_params = dict(self.searchcv.cv_results_["params"][best_idx])
        self.estimator.set_params(**best_params)
        scores_cv = cross_val_score(
            self.estimator,
            X,
            y=y,
            groups=groups,
            scoring=self.searchcv.scoring,
            cv=self.searchcv.cv,
            n_jobs=n_jobs,
        )
        mu = np.mean(scores_cv)
        #best_score = np.max(scores_cv)
        if verbose:
            print(f"\nSearching elapses {t1 - t0:.6f} seconds.\n")
            print(f"\nbest_params: {best_params}\n")
            print(f"\ncv scores：{scores_cv}\n")
            sigma = np.std(scores_cv)
            bounds_left = mu - 1.96 * sigma
            bounds_right = mu + 1.96 * sigma
            print(
                f"\ncv scores(95% CI): {mu:.6f} ± {1.96 * sigma:.6f} = "
                f"({bounds_left:.6f}, {bounds_right:.6f})\n"
            )
        self.estimator.fit(X, y)
        return best_params, mu, sigma


### 数据文件处理

In [ ]:
import shutil


def rand_resampling(X, y, class_range, choose_times):
    """
    :describe: 对分类模型原始样本进行随机重采样，提高样本数据量
    :author: anrui
    :date: 2022.04.29
    :modified: 2022.11.03
    :param X: 特征数据集
    :param y: 标签数据集
    :param class_range: 类别范围
    :param choose_times: 每种类别循环随机取的次数
    :return: X_new, y_new
    """
    X_new = []
    y_new = []
    for cc in class_range:
        X_sub = X[y == cc, :] # 一个类别所有特征数据
        y_sub = y[y == cc]    # 一个类别所有类别数据

        x_sub_tmp = X_sub.copy()         # 一个类别重采样后的样本
        y_sub_tmp = y_sub.copy()
        for i in range(0, choose_times):  # 多次随机采样
            seed(random_state + i)  # 初始种子一致，保证不同循序次数重采样结果不同
            for j in range(X_sub.shape[0] - 3):
                # 单个样本与其后的任意随机两个样本计算均值，得到新样本
                grp_X = X_sub[j, :].reshape(1, -1)  # 一个类别的单个样本
                the_last = X_sub[j + 1:-1, :]
                row_idx = np.arange(the_last.shape[0])
                shuffle(row_idx)
                choose_sample = the_last[row_idx[0:2], :]
                new_sample_tmp = np.append(grp_X, choose_sample, axis=0)  # 按行拼接
                new_sample = np.average(new_sample_tmp, axis=0)  # 按列求均值
                x_sub_tmp = np.r_[x_sub_tmp, new_sample.reshape(1, -1)]
                y_sub_tmp = np.r_[y_sub_tmp, cc]

        X_new += x_sub_tmp.tolist()
        y_new += y_sub_tmp.tolist()
        
    return np.array(X_new), np.array(y_new)


def rand_resampling_stratify_group(X, y, groups_train, choose_times):
    """
    :describe: 对分类模型原始样本进行随机重采样，提高样本数据量。分层分组重采样（本质上也为分层抽样）
    :author: anrui
    :date: 2022.10.15
    :param X: 特征数据集
    :param y: 标签数据集
    :param groups_train: 组号标签
    :param choose_times: 每种类别循环随机取的次数
    :return: X_new, y_new, group_new
    """
    X_new = []
    y_new = []
    group_new = []

    groups_list = np.unique(groups_train).tolist()

    for gg in groups_list:
        # 单个组进行抽样
        X_sub = X[groups_train == gg, :]             # 一个组号所有特征数据
        y_sub = y[groups_train == gg]                # 一个组号所有类别数据
        group_sub = groups_train[groups_train == gg] # 一个组号所有组号数据


        x_sub_tmp = X_sub.copy()         # 一个组号重采样后的样本
        y_sub_tmp = y_sub.copy()
        group_sub_tmp = group_sub.tolist().copy()

        for i in range(0, choose_times):  # 多次随机采样
            seed(random_state + i)  # 初始种子一致，保证不同循序次数重采样结果不同
            for j in range(X_sub.shape[0] - 3):
                # 单个样本与其后的任意随机两个样本计算均值，得到新样本
                grp_X = X_sub[j, :].reshape(1, -1)  # 一个类别的单个样本
                the_last = X_sub[j + 1:-1, :]
                row_idx = np.arange(the_last.shape[0])
                shuffle(row_idx)
                choose_sample = the_last[row_idx[0:2], :]
                new_sample_tmp = np.append(grp_X, choose_sample, axis=0)  # 按行拼接
                new_sample = np.average(new_sample_tmp, axis=0)  # 按列求均值
                x_sub_tmp = np.r_[x_sub_tmp, new_sample.reshape(1, -1)]
                y_sub_tmp = np.r_[y_sub_tmp, int(gg[:2])] # 组号为string类型，前两个字符为类别编码
                group_sub_tmp.append(gg)

        X_new += x_sub_tmp.tolist()
        y_new += y_sub_tmp.tolist()
        group_new += group_sub_tmp
        
    return np.array(X_new), np.array(y_new), np.asarray(group_new)
    
def grouped_avg(myArray, N=2):
    """
    :describe: 数组多行平均
    :author: anrui
    :date: 2022.04.20
    :param myArray: 原始二维数据
    :param N: 按行平均的行数
    :return: result
    """
    result = np.cumsum(myArray, 0)[N-1::N]/float(N)
    result[1:] = result[1:] - result[:-1]
    return result

def get_project_rootpath():
    """
    :describe: 获取项目根目录
    :author: anrui
    :date: 2022.05.19
    :param 
    :return: result
    """
    path = os.path.realpath(os.curdir)
    while (True):
        for subpath in os.listdir(path):
            if '.idea' in subpath or '.vscode' in subpath:  # 根目录下pycharm项目必然存在'.idea'文件，同理vscode项目必然存在'.vscode'文件
                return path
        path = os.path.dirname(path)

def mk_dir(path):
    # 去除首位空格
    path = path.strip()
    # 去除尾部 \ 符号
    path = path.rstrip("\\")
    # 判断路径是否存在
    # 存在    返回 True
    # 不存在  返回 False
    isExists = os.path.exists(path)
    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
        # 创建目录操作函数
        os.makedirs(path)
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        return False

def copy_dir(src_dir, des_dir):
    """
    :describe: 文件目录拷贝
    :author: anrui
    :date: 2022.09.24
    :param src_dir: 原始目录
    :param des_dir: 目标目录
    :return: 
    """
    if os.path.exists(des_dir):
        print("目标文件目录已存在，先删除")
        shutil.rmtree(des_dir)
    print("拷贝文件开始...")
    shutil.copytree(src_dir, des_dir)
    print("拷贝文件结束...")


# 数据准备

.、

# name建模

## 数据预处理

对建模目标进行编码：

In [ ]:
# 将建模target进行编码
from sklearn.preprocessing import LabelEncoder


df = df[f_names + [target]]

# 编码之前保存原始类别名称
level_names_tmp = np.unique(df[target])

le = LabelEncoder()
df[target] = le.fit_transform(df[[target]])
level_codes_tmp = np.unique(df[target])

# 建立等级名称与等级编码的字典
label_decoder = dict(zip(level_codes_tmp, level_names_tmp))
label_encoder = {val: key for key, val in label_decoder.items()}
print(label_decoder)
print(label_encoder)

# 所有等级名称
label_names = level_names_tmp.tolist()
class_range = range(0, len(label_decoder))

label_names_old = list(label_names).copy()
class_range_old = list(class_range).copy()
label_decoder_old = label_decoder.copy()

class_nums = len(class_range)

In [ ]:
#np.save(r'data/output/label_decoder.npy', label_decoder)

## 数据集划分

## 对训练集进行重采样

## 数据探索

## 机器学习

### 定义建模的自变量与因变量
建模所用的自变量为：

In [ ]:
names_X = f_names

样本数量：

In [ ]:
X_train = df_train[names_X].to_numpy()
y_train = df_train[target].to_numpy()

groups_train = df_train["组号"].to_numpy()

X_test = df_test[names_X].to_numpy()
y_test = df_test[target].to_numpy()

print("训练集样本大小：")
print(X_train.shape)
print(y_train.shape)

print("测试集样本大小：")
print(X_test.shape)
print(y_test.shape)

训练集样本大小：
(231920, 226)
(231920,)
测试集样本大小：
(77303, 226)
(77303,)


### 模型训练与表现

#### 模型得分汇总

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from tempfile import mkdtemp
from shutil import rmtree

# 所有选用的模型
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import PowerTransformer, QuantileTransformer


# 模型得分汇总
mdl_names = []
mdl_ct_types = []
base_estimators = []
best_params = []
cv_scores = []
cv_scores_std = []
mdl_times = []


def model_train(X_train, y_train, groups_train=None, method=None, ct_type=None):
    name = method
    
    t0 = time()
    if method == "Logistic Regression":
        classifier = LogisticRegression(random_state=random_state, max_iter=5000)
        param_grid = {
            'model__C': [0.01, 0.1, 1, 10, 100],
        }
    elif method == "QDA":
        classifier = QDA()
        param_grid = {
            'model__reg_param': np.logspace(-12, 1, 20),
            # 'spcprep__spectrum__kbest__k': range(4, 12),
            # 'spcprep__SNV_spectrum__kbest__k': range(4, 12),
        }
    elif method == "GaussianNB":
        classifier = GaussianNB()
        param_grid = {
            'model__var_smoothing': np.logspace(-10, 3, 20),
        }
    elif method == "SVMClassifier":
        classifier = SVC(probability=True) # with score_log
        param_grid = {
            #'model__kernel':['poly', 'rbf'],
            'model__C': [0.1, 1, 10, 100, 500],
            'model__gamma': [100, 10, 1, 0.1, 0.01],
        }
    elif method == "KNN":
        classifier = KNeighborsClassifier()
        param_grid = {
            'model__n_neighbors': np.arange(2,10),
        }
    elif method == "Decision Tree":
        classifier = DecisionTreeClassifier(
            random_state=random_state
        )
        param_grid = {
            'model__max_depth': [2, 5, 10, 20],
            'model__min_samples_leaf': [5, 10, 20, 50, 100],
            'model__criterion': ["gini", "entropy"]
        }
    elif method == "Random Forest":
        classifier = RandomForestClassifier(
            random_state=random_state,
        )
        param_grid = {
            'model__max_depth': range(3, 10),
            'model__min_samples_leaf': range(1, 6),
            'model__min_samples_split': range(2, 10),
        }
    elif method == "XGBoost":
        classifier = XGBClassifier(
            eval_metric='mlogloss',
            use_label_encoder=False,
        )
        param_grid = {
            'model__learning_rate': np.arange(0.01, 0.1, 0.01),
            'model__min_child_weight': [1, 5, 10],
            'model__gamma': [0.5, 1, 1.5, 2, 5],
            'model__subsample': [0.6, 0.8, 1.0],
            'model__colsample_bytree': [0.6, 0.8, 1.0],
            'model__max_depth': range(2, 10, 1),
        }
    elif method == "LightGBM":
        classifier = LGBMClassifier(
            boosting_type='gbdt',
            objective='multiclass',
            metric='multi_logloss',
            num_iterations=1000,
        )
        param_grid = {
            'model__learning_rate': np.arange(0.002, 0.02, 0.005),
            'model__num_leaves': range(30, 80, 10),
            #'model__reg_alpha': [0.1, 0.5],
            #'model__min_data_in_leaf': [30, 50, 100, 300, 400],
            #'model__lambda_l1': [0, 1, 1.5],
            #'model__lambda_l2': [0, 1],
        }
    else:
        print("no such method...")
        return

    spectrum_idxs = list(range(X_train.shape[1]))[:len(names_X):]
    cachedir = mkdtemp() # 创建缓存文件
        
    if ct_type == "SPC_Remain":
        ct = ColumnTransformer(
            transformers=[
                (
                    'SPC_Remain',
                    Pipeline(
                        steps=[
                            ('spc_remain', SPC_Remain()),
                            ('scale', StandardScaler()),
                            ('pca', PCA(n_components=0.9995,random_state=random_state)),
                        ]),
                    spectrum_idxs,
                ),
            ],
            remainder='passthrough',
        )
    elif ct_type == "CWT":
        ct = ColumnTransformer(
            transformers=[
                (
                    'CWT',
                    Pipeline(
                        steps=[
                            ('cwt', CWT(range(9, 10))),
                            ('scale', StandardScaler()),
                            ('pca', PCA(n_components=0.9995,random_state=random_state)),
                        ]),
                    spectrum_idxs,
                ),
            ],
            remainder='passthrough',
        )
    else:
        print("wrong ct_type...")
        return

    pipeline = Pipeline(
        steps=[
            ('spcprep', ct),
            ('model', classifier),
        ],
        memory=cachedir,
    )

    # 时间序列交叉验证
    custom_splitter = StratifiedGroupTimeSeriesSplit(X_train, y_train, groups_train, n_splits=2)

    searchcv = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=custom_splitter,
        scoring=scoring_log,
        verbose=0,
        refit=False,
        n_jobs=-1,
    )

    superestimator = SuperEstimator(searchcv)
    best_params, cv_score, cv_score_std = superestimator.search(X_train, y_train.astype('int'), groups=groups_train)
    mdl = superestimator.estimator

    ctime = round(time() - t0, 3)
    rmtree(cachedir, ignore_errors=True) # 删除缓存文件

    return ctime, name, ct_type, cv_score, cv_score_std, best_params, mdl

#### 模型训练及成绩

In [ ]:
model_list = ["Random Forest", "SVMClassifier"]#, "Logistic Regression", "Decision Tree", "GaussianNB", "KNN", "QDA", "KNN", "LightGBM", "XGBoost"]
ct_types = ["CWT"]

for model in model_list:
    for ct_type in ct_types:
        print(f"++++++++++++++++++++++++训练模型：{model}--{ct_type}++++++++++++++++++++++++\n")
        ctime, name, ct_type, cv_score, cv_score_std, best_param, mdl = model_train(X_train, y_train, groups_train, method=model, ct_type=ct_type)

        mdl_names.append(name)
        mdl_ct_types.append(ct_type)
        base_estimators.append((name, ct_type, mdl))
        best_params.append((name, best_param))
        cv_scores.append(cv_score)
        cv_scores_std.append(cv_score_std)
        mdl_times.append(ctime)

In [ ]:
mdls = pd.DataFrame(
    data=np.hstack((
        np.array(mdl_names).reshape(-1,1),
        np.array(mdl_ct_types).reshape(-1,1),
        np.array(cv_scores).reshape(-1,1),
        np.array(cv_scores_std).reshape(-1,1),
        np.array(mdl_times).reshape(-1,1),
    )), 
    columns=["mdl_names", "mdl_ct_types", "cv_score", "cv_scores_std", "model_time (s)"]
)
print("++++++++++++++++++++++++所有模型成绩++++++++++++++++++++++++")
mdls

### 模型选择

In [ ]:
from sklearn import set_config


set_config(display="diagram")
mdl = base_estimators[np.argmax(cv_scores)][2]
#mdl = base_estimators[2][2]
print("++++++++++++++++++++++++最佳模型++++++++++++++++++++++++")
mdl

模型持久化：

In [ ]:
# 保存模型
import joblib

mdl_path = r'data/output/model_.pkl'
joblib.dump(mdl, mdl_path)

展示transformer之后得到的特征名称：

In [ ]:
#mdl.named_steps['spcprep'].named_transformers_['PCA_MSC'].named_steps['PCA'].get_feature_names_out().tolist()

### 测试集的表现

#### 使用全部测试集进行模型验证

In [ ]:
cmshow(mdl, X_test, y_test, label_names)

## 使用保留下来的外部验证集验证模型

In [ ]:
df_val.info()

### 外测集的表现

#### 使用全部外测集进行模型验证

In [ ]:
# 加载和使用模型
mdl = joblib.load(r'data/output/model.pkl')

In [ ]:
X_val, y_val= df_val[names_X].to_numpy(), df_val[target].to_numpy()

cmshow(mdl, X_val, y_val, label_names)

# 结论

* ./
* ,.